In [ ]:
#def unpairedttest(ar1,ar2):
#    npar1 = np.array(ar1)
#    npar2 = np.array(ar2)
#    return np.abs(npar1.mean()-npar2.mean())/np.sqrt((npar1.var()/npar1.shape[0])+(npar2.var()/npar2.shape[0]))

In [1]:
from model import UNet
import helper_functions as hp
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import warnings
warnings.simplefilter("ignore")
import numpy as np
import os
import scipy.io
import json
from skimage import metrics
import scipy.stats

In [2]:
def pixelwiseFPFNTP(pred,gold):
    TP = np.where(gold+pred==2,1,0).sum()
    FP = np.where(pred-gold==1,1,0).sum()
    FN = np.where(gold-pred==1,1,0).sum()

    return TP, FP, FN

def getPrecRecallFScore(TP, FP, FN):
    if (TP + FP) > 0:
        precision = TP / (TP + FP)
    else:
        precision = 0
    if (TP + FN) > 0:    
        recall = TP / (TP + FN)
    else:
        recall = 0
    if (precision+recall) > 0:    
        f_score = (2*precision*recall)/(precision+recall)
    else:
        f_score = 0
        
    return precision, recall, f_score 

def pixelwise(pred,gold):
    
    TP, FP, FN = pixelwiseFPFNTP(pred,gold)

    return getPrecRecallFScore(TP, FP, FN)

def IoU(prediction, goldB, threshold):
    goldB_cc = cv2.connectedComponents(goldB)
    pred_cc = cv2.connectedComponents(prediction)
    TP = 0
    for i in range(1,goldB_cc[0]+1):
        goldB_inst = np.where(goldB_cc[1]==i,1,0)
        
        inter_num = 0
        for j in range(1,pred_cc[0]+1):
            pred_inst = np.where(pred_cc[1]==j,1,0)
            union = goldB_inst + pred_inst
            if union.max() == 2:
                intersection = np.where(union==2,1,0)
                inter_num_curr = intersection.sum()
                if inter_num_curr > inter_num:
                    inter_num = inter_num_curr
                    union_chosen = union
                    
        if inter_num > 0:
            union_chosen = np.where(union_chosen>0,1,0)
            union_num = union_chosen.sum()
            if inter_num/union_num >= threshold:
                TP = TP + 1
                
    segmented = pred_cc[0] - 1
    positives = goldB_cc[0] - 1
    if segmented > 0:
        precision = TP / segmented
    else:
        precision = 0
    recall = TP / positives
    if precision+recall > 0:
        f_score = round(2*((precision*recall)/(precision+recall)),2)
    else:
        f_score = 0      
        
    return precision, recall, f_score

def get_weighted_hausdorff_dist(pred,gold):
    flag = False;
    if pred.sum() == 0:
        pred[0,:] = 1; pred[255,:] = 1; pred[:,0] = 1; pred[:,255] = 1;  
        flag = True
    gold_cc = cv2.connectedComponents(gold)
    pred_cc = cv2.connectedComponents(pred)
    return 0.5 * (wdh_one(pred_cc, gold_cc, pred,gold, flag) + wdh_one(gold_cc, pred_cc, gold,pred,flag))

def wdh_one(first_cc,second_cc,first,second, flag):
    whd = 0
    for i in range(1,first_cc[0]):
        first_inst = np.where(first_cc[1]==i,1,0)

        weight = first_inst.sum() / first.sum()

        if (first_inst * second).sum() > 0:
            intersection = 0
            for j in range(1,second_cc[0]):
                second_inst = np.where(second_cc[1]==j,1,0)
                intersection_new = (first_inst * second_inst).sum()
                if intersection_new > intersection:
                    hd = metrics.hausdorff_distance(first_inst==1, second_inst==1)
                    intersection = intersection_new
        else:
            if flag:
                hd = 0
                for j in range(1,second_cc[0]):
                    second_inst = np.where(second_cc[1]==j,1,0) 
                    hd_new = metrics.hausdorff_distance(first_inst==1, second_inst==1)
                    if hd_new > hd:
                        hd = hd_new                  
            else:
                hd = np.inf
                for j in range(1,second_cc[0]):
                    second_inst = np.where(second_cc[1]==j,1,0) 
                    hd_new = metrics.hausdorff_distance(first_inst==1, second_inst==1)
                    if hd_new < hd:
                        hd = hd_new  
    
        whd = whd + (weight * hd)   
    return whd        

In [3]:
first_out_channel = 32
gold_folder = 'data/goldsBinaryAll'
mn_base = ['oneTaskMaxPoolAll2/weights/allWeightsB2.pth','oneTaskMaxPoolAll2/weights/allWeightsB2_2.pth','oneTaskMaxPoolAll2/weights/allWeightsB2_3.pth','oneTaskMaxPoolAll2/weights/allWeightsB2_4.pth','oneTaskMaxPoolAll2/weights/allWeightsB2_5.pth']
mn_our = ['oneTaskMaxPoolAll_bd_25_tmp3/weights/allWeightsB2.pth','oneTaskMaxPoolAll_bd_25_tmp3/weights/allWeightsB2_2.pth','oneTaskMaxPoolAll_bd_25_tmp3/weights/allWeightsB2_3.pth','oneTaskMaxPoolAll_bd_25_tmp3/weights/allWeightsB2_4.pth','oneTaskMaxPoolAll_bd_25_tmp3/weights/allWeightsB2_5.pth']
mn_neurips = ['NeurIPSAll_25/weights/allWeightsB2.pth','NeurIPSAll_25_2/weights/allWeightsB2_2.pth','NeurIPSAll_25_3/weights/allWeightsB2_3.pth','NeurIPSAll_25_4/weights/allWeightsB2_4.pth','NeurIPSAll_25_5/weights/allWeightsB2_5.pth']
fourFolder = ['FourierNet/results_1/input-cascade_oct_L4_F16_fold2_run1/','FourierNet/results_2/input-cascade_oct_L4_F16_fold2_run1/','FourierNet/results_3/input-cascade_oct_L4_F16_fold2_run1/','FourierNet/results_4/input-cascade_oct_L4_F16_fold2_run1/','FourierNet/results_5/input-cascade_oct_L4_F16_fold2_run1/']

In [4]:
import os
import json

with open('data/vesseltypes.json') as f:
    fileTypesDict = json.load(f)
    
image_names = os.listdir('data/bucket2/ts')

typ = []

basePrecisionPixel = []
baseRecallPixel = []
baseFscorePixel = []
ourPrecisionPixel = []
ourRecallPixel = []
ourFscorePixel = []
neurPrecisionPixel = []
neurRecallPixel = []
neurFscorePixel = []
fourPrecisionPixel = []
fourRecallPixel = []
fourFscorePixel = []
basePrecisionIoU = []
baseRecallIoU = []
baseFscoreIoU = []
ourPrecisionIoU = []
ourRecallIoU = []
ourFscoreIoU = []
neurPrecisionIoU = []
neurRecallIoU = []
neurFscoreIoU = []
fourPrecisionIoU = []
fourRecallIoU = []
fourFscoreIoU = []
baseHaus = []
ourHaus = []
neurHaus = []
fourHaus = []

for i in range(5):
    for name in image_names:    
        typ.append(fileTypesDict[name])

        tst_im_name = 'data/bucket2/ts/' + name
        gold_im_name = 'data/goldsBinaryAll/' + name.split('.')[0] + '.png'

        _, _, pred_base, gold = hp.test(1, first_out_channel, mn_base[i], tst_im_name, gold_im_name)

        precision, recall, f_score = pixelwise(pred_base, gold)
        basePrecisionPixel.append(precision)
        baseRecallPixel.append(recall)
        baseFscorePixel.append(f_score)

        precision, recall, f_score = IoU(pred_base, gold, 0.5)
        basePrecisionIoU.append(precision)
        baseRecallIoU.append(recall)
        baseFscoreIoU.append(f_score)

        whd = get_weighted_hausdorff_dist(pred_base, gold)
        baseHaus.append(whd)

        _, _, pred_our, _ = hp.test(1, first_out_channel, mn_our[i], tst_im_name, gold_im_name)

        precision, recall, f_score = pixelwise(pred_our, gold)
        ourPrecisionPixel.append(precision)
        ourRecallPixel.append(recall)
        ourFscorePixel.append(f_score)

        precision, recall, f_score = IoU(pred_our, gold, 0.5)
        ourPrecisionIoU.append(precision)
        ourRecallIoU.append(recall)
        ourFscoreIoU.append(f_score)

        whd = get_weighted_hausdorff_dist(pred_our, gold)
        ourHaus.append(whd)

        _, _, pred_neurips, _ = hp.test(1, first_out_channel, mn_neurips[i], tst_im_name, gold_im_name)  

        precision, recall, f_score = pixelwise(pred_neurips, gold)
        neurPrecisionPixel.append(precision)
        neurRecallPixel.append(recall)
        neurFscorePixel.append(f_score)

        precision, recall, f_score = IoU(pred_neurips, gold, 0.5)
        neurPrecisionIoU.append(precision)
        neurRecallIoU.append(recall)
        neurFscoreIoU.append(f_score)

        whd = get_weighted_hausdorff_dist(pred_neurips, gold)
        neurHaus.append(whd)

        pred_four = cv2.imread(fourFolder[i] + name.split('.')[0] + '.png',cv2.IMREAD_GRAYSCALE)

        precision, recall, f_score = pixelwise(pred_four, gold)
        fourPrecisionPixel.append(precision)
        fourRecallPixel.append(recall)
        fourFscorePixel.append(f_score)

        precision, recall, f_score = IoU(pred_four, gold, 0.5)
        fourPrecisionIoU.append(precision)
        fourRecallIoU.append(recall)
        fourFscoreIoU.append(f_score)

        whd = get_weighted_hausdorff_dist(pred_four, gold)
        fourHaus.append(whd)   

In [5]:
def pairedttest(ar1,ar2):#we should use this because we take measurement on prediction
    arr = np.array(ar1) - np.array(ar2)
    return arr.mean()/(arr.std()/np.sqrt(arr.shape[0])),np.array(ar1).mean(),np.array(ar2).mean()

In [6]:
compare_list = [['ourPrecisionPixel',ourPrecisionPixel, basePrecisionPixel],
['ourRecallPixel',ourRecallPixel, baseRecallPixel],
['ourFscorePixel',ourFscorePixel, baseFscorePixel],
['neurPrecisionPixel',neurPrecisionPixel, basePrecisionPixel],
['neurRecallPixel',neurRecallPixel, baseRecallPixel],
['neurFscorePixel',neurFscorePixel, baseFscorePixel],
['fourPrecisionPixel',fourPrecisionPixel, basePrecisionPixel],
['fourRecallPixel',fourRecallPixel, baseRecallPixel],
['fourFscorePixel',fourFscorePixel, baseFscorePixel],
['ourPrecisionIoU',ourPrecisionIoU, basePrecisionIoU],
['ourRecallIoU',ourRecallIoU, baseRecallIoU],
['ourFscoreIoU',ourFscoreIoU, baseFscoreIoU],
['neurPrecisionIoU',neurPrecisionIoU, basePrecisionIoU],
['neurRecallIoU',neurRecallIoU, baseRecallIoU],
['neurFscoreIoU',neurFscoreIoU, baseFscoreIoU],
['fourPrecisionIoU',fourPrecisionIoU, basePrecisionIoU],
['fourRecallIoU',fourRecallIoU, baseRecallIoU],
['fourFscoreIoU',fourFscoreIoU, baseFscoreIoU],
['ourHaus',ourHaus, baseHaus],
['neurHaus',neurHaus, baseHaus],
['fourHaus',fourHaus, baseHaus]]

In [9]:
for item in compare_list:
    ar1 = [item[1][i] for i in range(len(item[1])) if typ[i] == 'f']
    ar2 = [item[2][i] for i in range(len(item[2])) if typ[i] == 'f']
    print(item[0], 'big')
    #print(pairedttest(ar1, ar2))
    print(scipy.stats.ttest_rel(ar1, ar2))
    ar1 = [item[1][i] for i in range(len(item[1])) if typ[i] != 'f']
    ar2 = [item[2][i] for i in range(len(item[2])) if typ[i] != 'f']
    print(item[0], 'other')
    #print(pairedttest(ar1, ar2))
    print(scipy.stats.ttest_rel(ar1, ar2))
    print(item[0], 'all')
    #print(pairedttest(item[1], item[2]))
    print(scipy.stats.ttest_rel(item[1], item[2]))

ourPrecisionPixel big
Ttest_relResult(statistic=6.540115771040568, pvalue=7.058337493267227e-11)
ourPrecisionPixel other
Ttest_relResult(statistic=3.9364530058489424, pvalue=8.420787401355987e-05)
ourPrecisionPixel all
Ttest_relResult(statistic=5.885056068921437, pvalue=4.158886609941077e-09)
ourRecallPixel big
Ttest_relResult(statistic=4.307539293819047, pvalue=1.6970358091677647e-05)
ourRecallPixel other
Ttest_relResult(statistic=10.186692023162635, pvalue=4.6915440883457126e-24)
ourRecallPixel all
Ttest_relResult(statistic=11.015371501053117, pvalue=5.404064825211508e-28)
ourFscorePixel big
Ttest_relResult(statistic=6.014990051098841, pvalue=1.9878073466897352e-09)
ourFscorePixel other
Ttest_relResult(statistic=9.143232780135886, pvalue=9.73618870949221e-20)
ourFscorePixel all
Ttest_relResult(statistic=10.767766120506414, pvalue=7.841099575191193e-27)
neurPrecisionPixel big
Ttest_relResult(statistic=1.0708032543108505, pvalue=0.2843331567940925)
neurPrecisionPixel other
Ttest_relRes